# Brain Tumor Classification

https://github.com/aksh-ai/neuralBlack/blob/master/torch_brain_tumor_classifier.ipynb

In [ ]:
!pip install numpy pandas matplotlib scikit-learn scikit-image

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install scipy seaborn

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
# from preprocess_new import load_datasets

In [ ]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms, models
from torchvision.utils import make_grid
from sklearn.metrics import confusion_matrix, classification_report
import os
import random
import numpy as np
import pandas as pd
import pickle
import time
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
torch.__version__

'1.13.0+cu116'

In [ ]:
!nvidia-smi

Sat Dec 10 05:52:36 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   69C    P0    29W /  70W |   4564MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
def get_data_loader(data_dir="/content/drive/MyDrive/Training", batch_size=32, train=True):
    """
    Define the way we compose the batch dataset including the augmentation for increasing the number of data
    and return the augmented batch-dataset
    :param data_dir: root directory where the either train or test dataset is
    :param batch_size: size of the batch
    :param train: true if current phase is training, else false
    :return: augmented batch dataset
    """

    # define how we augment the data for composing the batch-dataset in train and test step
    transform = {
        'train': transforms.Compose([
            transforms.Resize([224, 224]),  # Resizing the image as the VGG only take 224 x 244 as input size
            transforms.RandomHorizontalFlip(),  # Flip the data horizontally
            # TODO if it is needed, add the random crop
            transforms.ToTensor(),
            # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ]),
        'test': transforms.Compose([
            transforms.Resize([224, 224]),
            transforms.RandomHorizontalFlip(),
            transforms.ToTensor(),
            # transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
        ])
    }

    # ImageFloder with root directory and defined transformation methods for batch as well as data augmentation
    data = torchvision.datasets.ImageFolder(root=data_dir, transform=transform['train'] if train else transform['test'])
    data_loader = torch.utils.data.DataLoader(dataset=data, batch_size=batch_size, shuffle=True, num_workers=4)
    return data_loader

In [ ]:
# test_data = np.load('drive/MyDrive/archive/test_0_new.npz')
# test_data = np.load('/content/drive/MyDrive/test_0_new.npz')
test_loader = get_data_loader(data_dir="/content/drive/MyDrive/Testing", batch_size=32, train=False)
train_loader = get_data_loader(data_dir="/content/drive/MyDrive/Training", batch_size=32, train=True)

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


In [ ]:
device_name = "cuda:0" if torch.cuda.is_available() else "cpu"
device = torch.device(device_name)

# instantiate transfer learning model
resnet_model = models.resnet50(pretrained=True)

# set all paramters as trainable
for param in resnet_model.parameters():
    param.requires_grad = True

# get input of fc layer
n_inputs = resnet_model.fc.in_features

# redefine fc layer / top layer/ head for our classification problem
resnet_model.fc = nn.Sequential(nn.Linear(n_inputs, 2048),
                                nn.SELU(),
                                nn.Dropout(p=0.4),
                                nn.Linear(2048, 2048),
                                nn.SELU(),
                                nn.Dropout(p=0.4),
                                nn.Linear(2048, 4),
                                nn.LogSigmoid())

# set all paramters of the model as trainable
for name, child in resnet_model.named_children():
    for name2, params in child.named_parameters():
        params.requires_grad = True

# set model to run on GPU or CPU absed on availibility
resnet_model.to(device)

# print the trasnfer learning NN model's architecture
resnet_model

/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.8/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [ ]:
# loss function
# if GPU is available set loss function to use GPU
criterion = nn.CrossEntropyLoss().to(device)

# optimizer
optimizer = torch.optim.SGD(resnet_model.parameters(), momentum=0.9, lr=3e-4)

# number of training iterations
epochs = 30

# empty lists to store losses and accuracies
train_losses = []
test_losses = []
train_correct = []
test_correct = []


def save_checkpoint(state, is_best, filename='/content/drive/My Drive/bt_resnet50_ckpt_v2.pth.tar'):
    torch.save(state, filename)


In [ ]:
# train_gen = train_loader
train_gen = train_loader
valid_gen = test_loader

# set training start time
start_time = time.time()

# set best_prec loss value as 2 for checkpoint threshold
best_prec1 = 2

# empty batch variables
b = None
train_b = None
test_b = None

# start training
for i in range(epochs):
    # empty training correct and test correct counter as 0 during every iteration
    trn_corr = 0
    tst_corr = 0
    total = 0

    # set epoch's starting time
    e_start = time.time()

    # train in batches
    for b, (X, y) in enumerate(train_gen):
        # set label as cuda if device is cuda
        print(f"Epoch: {i}, Batch: {b}, Device: {device}")
        X, y = X.to(device), y.to(device)

        # forward pass image sample
        y_pred = resnet_model(X.view(-1, 3, 224, 224))
        # calculate loss
        loss = criterion(y_pred.float(), y.long())
        # loss = criterion(y_pred.float(), torch.argmax(y.view(32, 4), dim=1).long())

        # get argmax of predicted tensor, which is our label
        predicted = torch.argmax(y_pred, dim=1).data
        # if predicted label is correct as true label, calculate the sum for samples
        batch_corr = (predicted == y).sum()
        # batch_corr = (predicted == torch.argmax(y.view(32, 4), dim=1)).sum()
        # increment train correct with correcly predicted labels per batch
        trn_corr += batch_corr
        # increment sample size
        total += y.nelement()

        # set optimizer gradients to zero
        optimizer.zero_grad()
        # back propagate with loss
        loss.backward()
        # perform optimizer step
        optimizer.step()

    # set epoch's end time
    e_end = time.time()
    # print training metrics
    print \
        (f'Epoch {(i + 1)} Batch {(b + 1) * 4}\nAccuracy: {trn_corr.item() * 100 / (total):2.2f} %  Loss: {loss.item():2.4f}  Duration: {((e_end - e_start) / 60):.2f} minutes')  # 4 images per batch * 8 augmentations per image * batch length

    # some metrics storage for visualization
    train_b = b
    train_losses.append(loss)
    train_correct.append(trn_corr)

    X, y = None, None

    y_true = list()
    y_pred = list()

    total = 0

    # validate using validation generator
    # do not perform any gradient updates while validation
    with torch.no_grad():
        for b, (X, y) in enumerate(valid_gen):
            # set label as cuda if device is cuda
            X, y = X.to(device), y.to(device)

            # forward pass image
            y_val = resnet_model(X.view(-1, 3, 224, 224))

            # get argmax of predicted tensor, which is our label
            predicted = torch.argmax(y_val, dim=1).data

            y_true.append(y)
            y_pred.append(predicted)

            # increment test correct with correcly predicted labels per batch
            tst_corr += (predicted == y).sum()

            # increment sample size
            total += y.nelement()

    y_true = torch.cat(y_true, 0).cpu().numpy()
    y_pred = torch.cat(y_pred, 0).cpu().numpy()
    y_true[y_true == 0] = 9
    y_pred[y_pred == 0] = 9
    y_true[y_true == 2] = 0
    y_pred[y_pred == 2] = 0
    y_true[y_true == 1] = 2
    y_pred[y_pred == 1] = 2
    y_true[y_true == 9] = 1
    y_pred[y_pred == 9] = 1
    print(confusion_matrix(y_true, y_pred))
    print(classification_report(y_true, y_pred))

    # get loss of validation set
    loss = criterion(y_val.float(), y.long())
    # print validation metrics
    print(f'Validation Accuracy {tst_corr.item() * 100 / (total):2.2f} Validation Loss: {loss.item():2.4f}\n')

    # if current validation loss is less than previous iteration's validatin loss create and save a checkpoint
    is_best = loss < best_prec1
    best_prec1 = min(loss, best_prec1)
    save_checkpoint({
        'epoch': i + 1,
        'state_dict': resnet_model.state_dict(),
        'best_prec1': best_prec1,
    }, is_best)

    # some metrics storage for visualization
    test_b = b
    test_losses.append(loss)
    test_correct.append(tst_corr)

# set total training's end time
end_time = time.time() - start_time

# print training summary
print("\nTraining Duration {:.2f} minutes".format(end_time / 60))
print("GPU memory used : {} kb".format(torch.cuda.memory_allocated()))
print("GPU memory cached : {} kb".format(torch.cuda.memory_cached()))


Epoch: 0, Batch: 0, Device: cuda:0
Epoch: 0, Batch: 1, Device: cuda:0
Epoch: 0, Batch: 2, Device: cuda:0
Epoch: 0, Batch: 3, Device: cuda:0
Epoch: 0, Batch: 4, Device: cuda:0
Epoch: 0, Batch: 5, Device: cuda:0
Epoch: 0, Batch: 6, Device: cuda:0
Epoch: 0, Batch: 7, Device: cuda:0
Epoch: 0, Batch: 8, Device: cuda:0
Epoch: 0, Batch: 9, Device: cuda:0
Epoch: 0, Batch: 10, Device: cuda:0
Epoch: 0, Batch: 11, Device: cuda:0
Epoch: 0, Batch: 12, Device: cuda:0
Epoch: 0, Batch: 13, Device: cuda:0
Epoch: 0, Batch: 14, Device: cuda:0
Epoch: 0, Batch: 15, Device: cuda:0
Epoch: 0, Batch: 16, Device: cuda:0
Epoch: 0, Batch: 17, Device: cuda:0
Epoch: 0, Batch: 18, Device: cuda:0
Epoch: 0, Batch: 19, Device: cuda:0
Epoch: 0, Batch: 20, Device: cuda:0
Epoch: 0, Batch: 21, Device: cuda:0
Epoch: 0, Batch: 22, Device: cuda:0
Epoch: 0, Batch: 23, Device: cuda:0
Epoch: 0, Batch: 24, Device: cuda:0
Epoch: 0, Batch: 25, Device: cuda:0
Epoch: 0, Batch: 26, Device: cuda:0
Epoch: 0, Batch: 27, Device: cuda:0
Ep

/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.

Epoch: 1, Batch: 0, Device: cuda:0
Epoch: 1, Batch: 1, Device: cuda:0
Epoch: 1, Batch: 2, Device: cuda:0
Epoch: 1, Batch: 3, Device: cuda:0
Epoch: 1, Batch: 4, Device: cuda:0
Epoch: 1, Batch: 5, Device: cuda:0
Epoch: 1, Batch: 6, Device: cuda:0
Epoch: 1, Batch: 7, Device: cuda:0
Epoch: 1, Batch: 8, Device: cuda:0
Epoch: 1, Batch: 9, Device: cuda:0
Epoch: 1, Batch: 10, Device: cuda:0
Epoch: 1, Batch: 11, Device: cuda:0
Epoch: 1, Batch: 12, Device: cuda:0
Epoch: 1, Batch: 13, Device: cuda:0
Epoch: 1, Batch: 14, Device: cuda:0
Epoch: 1, Batch: 15, Device: cuda:0
Epoch: 1, Batch: 16, Device: cuda:0
Epoch: 1, Batch: 17, Device: cuda:0
Epoch: 1, Batch: 18, Device: cuda:0
Epoch: 1, Batch: 19, Device: cuda:0
Epoch: 1, Batch: 20, Device: cuda:0
Epoch: 1, Batch: 21, Device: cuda:0
Epoch: 1, Batch: 22, Device: cuda:0
Epoch: 1, Batch: 23, Device: cuda:0
Epoch: 1, Batch: 24, Device: cuda:0
Epoch: 1, Batch: 25, Device: cuda:0
Epoch: 1, Batch: 26, Device: cuda:0
Epoch: 1, Batch: 27, Device: cuda:0
Ep

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 2, Batch: 0, Device: cuda:0
Epoch: 2, Batch: 1, Device: cuda:0
Epoch: 2, Batch: 2, Device: cuda:0
Epoch: 2, Batch: 3, Device: cuda:0
Epoch: 2, Batch: 4, Device: cuda:0
Epoch: 2, Batch: 5, Device: cuda:0
Epoch: 2, Batch: 6, Device: cuda:0
Epoch: 2, Batch: 7, Device: cuda:0
Epoch: 2, Batch: 8, Device: cuda:0
Epoch: 2, Batch: 9, Device: cuda:0
Epoch: 2, Batch: 10, Device: cuda:0
Epoch: 2, Batch: 11, Device: cuda:0
Epoch: 2, Batch: 12, Device: cuda:0
Epoch: 2, Batch: 13, Device: cuda:0
Epoch: 2, Batch: 14, Device: cuda:0
Epoch: 2, Batch: 15, Device: cuda:0
Epoch: 2, Batch: 16, Device: cuda:0
Epoch: 2, Batch: 17, Device: cuda:0
Epoch: 2, Batch: 18, Device: cuda:0
Epoch: 2, Batch: 19, Device: cuda:0
Epoch: 2, Batch: 20, Device: cuda:0
Epoch: 2, Batch: 21, Device: cuda:0
Epoch: 2, Batch: 22, Device: cuda:0
Epoch: 2, Batch: 23, Device: cuda:0
Epoch: 2, Batch: 24, Device: cuda:0
Epoch: 2, Batch: 25, Device: cuda:0
Epoch: 2, Batch: 26, Device: cuda:0
Epoch: 2, Batch: 27, Device: cuda:0
Ep

...

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 28, Batch: 0, Device: cuda:0
Epoch: 28, Batch: 1, Device: cuda:0
Epoch: 28, Batch: 2, Device: cuda:0
Epoch: 28, Batch: 3, Device: cuda:0
Epoch: 28, Batch: 4, Device: cuda:0
Epoch: 28, Batch: 5, Device: cuda:0
Epoch: 28, Batch: 6, Device: cuda:0
Epoch: 28, Batch: 7, Device: cuda:0
Epoch: 28, Batch: 8, Device: cuda:0
Epoch: 28, Batch: 9, Device: cuda:0
Epoch: 28, Batch: 10, Device: cuda:0
Epoch: 28, Batch: 11, Device: cuda:0
Epoch: 28, Batch: 12, Device: cuda:0
Epoch: 28, Batch: 13, Device: cuda:0
Epoch: 28, Batch: 14, Device: cuda:0
Epoch: 28, Batch: 15, Device: cuda:0
Epoch: 28, Batch: 16, Device: cuda:0
Epoch: 28, Batch: 17, Device: cuda:0
Epoch: 28, Batch: 18, Device: cuda:0
Epoch: 28, Batch: 19, Device: cuda:0
Epoch: 28, Batch: 20, Device: cuda:0
Epoch: 28, Batch: 21, Device: cuda:0
Epoch: 28, Batch: 22, Device: cuda:0
Epoch: 28, Batch: 23, Device: cuda:0
Epoch: 28, Batch: 24, Device: cuda:0
Epoch: 28, Batch: 25, Device: cuda:0
Epoch: 28, Batch: 26, Device: cuda:0
Epoch: 28, 

/usr/local/lib/python3.8/dist-packages/torch/utils/data/dataloader.py:554: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Epoch: 29, Batch: 0, Device: cuda:0
Epoch: 29, Batch: 1, Device: cuda:0
Epoch: 29, Batch: 2, Device: cuda:0
Epoch: 29, Batch: 3, Device: cuda:0
Epoch: 29, Batch: 4, Device: cuda:0
Epoch: 29, Batch: 5, Device: cuda:0
Epoch: 29, Batch: 6, Device: cuda:0
Epoch: 29, Batch: 7, Device: cuda:0
Epoch: 29, Batch: 8, Device: cuda:0
Epoch: 29, Batch: 9, Device: cuda:0
Epoch: 29, Batch: 10, Device: cuda:0
Epoch: 29, Batch: 11, Device: cuda:0
Epoch: 29, Batch: 12, Device: cuda:0
Epoch: 29, Batch: 13, Device: cuda:0
Epoch: 29, Batch: 14, Device: cuda:0
Epoch: 29, Batch: 15, Device: cuda:0
Epoch: 29, Batch: 16, Device: cuda:0
Epoch: 29, Batch: 17, Device: cuda:0
Epoch: 29, Batch: 18, Device: cuda:0
Epoch: 29, Batch: 19, Device: cuda:0
Epoch: 29, Batch: 20, Device: cuda:0
Epoch: 29, Batch: 21, Device: cuda:0
Epoch: 29, Batch: 22, Device: cuda:0
Epoch: 29, Batch: 23, Device: cuda:0
Epoch: 29, Batch: 24, Device: cuda:0
Epoch: 29, Batch: 25, Device: cuda:0
Epoch: 29, Batch: 26, Device: cuda:0
Epoch: 29, 

/usr/local/lib/python3.8/dist-packages/torch/cuda/memory.py:395: FutureWarning: torch.cuda.memory_cached has been renamed to torch.cuda.memory_reserved
  warnings.warn(
